# **Lab-10-2 Mnist CNN**

## **학습 단계(code 기준)**

1. 라이브러리 가져오고 (torch, torchvision, matplotlib 같은것들)

2. GPU 사용 설정 하고 random value를 위한 seed 설정!
(device 미리 설정, 그걸 이용해서 torch가 CUDA를 사용 가능할 때는 CUDA이용, 그렇지 않을경우 CPU이용함을 의미) 

3. 학습에 사용되는 parameter 설정!(learning_rate, training_epochs, batch_size, etc)

4. 데이터셋을 가져오고 (학습에 쓰기 편하게) loader 만들기

5. 학습 모델 만들기( class CNN(torch.nn.Module) )

6. Loss function (Criterion)을 선택하고 최적화 도구 선택(optimizer)

7. 모델 학습 및 loss check(Criterion의 output)

8. 학습된 모델의 성능을 확인한다.

## **직접 실습할 CNN 구조**

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FUAKFa%2FbtqCuZ56rnC%2FLeJkpv6NUIAx3YBC8siqO1%2Fimg.png)




In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [0]:
# setting device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [0]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [0]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [0]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [0]:
# instantiate CNN model
model = CNN().to(device)

In [54]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [0]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device) # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [56]:
# train my model
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    # image is already size of (28x28), no reshape
    # label is not one-hot encoded
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.224006683
[Epoch:    2] cost = 0.062186949
[Epoch:    3] cost = 0.0449030139
[Epoch:    4] cost = 0.0355709828
[Epoch:    5] cost = 0.0290450025
[Epoch:    6] cost = 0.0248527844
[Epoch:    7] cost = 0.0207189098
[Epoch:    8] cost = 0.0181982815
[Epoch:    9] cost = 0.0153046707
[Epoch:   10] cost = 0.0124179339
[Epoch:   11] cost = 0.0105423154
[Epoch:   12] cost = 0.00991860125
[Epoch:   13] cost = 0.00894770492
[Epoch:   14] cost = 0.0071221008
[Epoch:   15] cost = 0.00588585297
Learning Finished!


In [57]:
# Test model and check accuracy
with torch.no_grad(): # 학습을 안한다는 의미
  X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device) # 한번에 집어넣기 위해 펼침
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

Accuracy: 0.9883000254631042
